In [1]:
import pandas as pd
import numpy as np
import sklearn
from xgboost import XGBClassifier
import xgboost
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pickle
from sklearn.utils import shuffle

In [2]:
df = pd.read_csv('../data/trainWithMean.csv',nrows=1)


srch_id                             int64
date_time                          object
site_id                             int64
visitor_location_country_id         int64
visitor_hist_starrating           float64
visitor_hist_adr_usd              float64
prop_country_id                     int64
prop_id                             int64
prop_starrating                     int64
prop_review_score                 float64
prop_brand_bool                     int64
prop_location_score1              float64
prop_location_score2              float64
prop_log_historical_price         float64
position                            int64
price_usd                         float64
promotion_flag                      int64
srch_destination_id                 int64
srch_length_of_stay                 int64
srch_booking_window                 int64
srch_adults_count                   int64
srch_children_count                 int64
srch_room_count                     int64
srch_saturday_night_bool          

In [2]:
#Don't use datetime, Don't use categorical features, xgboost doesn't those, first one hot encode
featuresToUse = ['visitor_hist_starrating', 'visitor_hist_adr_usd',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'comp1_rate', 'comp1_inv',
                 'prop_starrating_mean',
       'prop_review_score_mean', 'prop_brand_bool_mean',
       'prop_location_score1_mean', 'prop_location_score2_mean',
       'prop_log_historical_price_mean']
labelToPredict = 'click_bool'
neededVars = ['srch_id','booking_bool']
nameOfModel = "clickingModel1may.model"

In [3]:
df = pd.read_csv('../data/trainWithMean.csv', delimiter=",",usecols=featuresToUse+[labelToPredict]+neededVars)
df = shuffle(df)
dfNeeded= df[neededVars+[labelToPredict]]
Y = df[labelToPredict]
df.drop(neededVars+[labelToPredict],axis=1,inplace=True)

Lets train it on clicking

In [4]:
n = 30
numOfRowsTest = (int(len(df)*(n/100)))
X_test = df.tail(numOfRowsTest)
Y_test = Y[-numOfRowsTest:]
dfNeeded_test = dfNeeded.tail(numOfRowsTest)

dfNeeded.drop(dfNeeded.tail(numOfRowsTest).index,inplace=True)
df.drop(df.tail(numOfRowsTest).index,inplace=True)
y_train = Y[:(len(Y)-numOfRowsTest)]

E:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [5]:
# construct xgboost.DMatrix from numpy array, treat -999.0 as missing value
#xgmat = xgboost.DMatrix( df, label=y_train)
xgmat = xgboost.DMatrix( df, label=y_train)
# setup parameters for xgboost
param = {}
# use logistic regression loss, use raw prediction before logistic transformation
# since we only need the rank
param['objective'] = 'reg:logistic'
# scale weight of positive examples
param['scale_pos_weight'] = sum(y_train==0)/sum(y_train==1)
param['eta'] = 0.1
param['max_depth'] = 6
param['eval_metric'] = 'auc'
param['silent'] = 0
param['nthread'] = -1

# you can directly throw param in, though we want to watch multiple metrics here
plst = list(param.items())

watchlist = [ (xgmat,'train') ]
# boost 120 trees
num_round = 120
print ('loading data end, start to boost trees')
bst = xgboost.train( plst, xgmat, num_round, watchlist ,early_stopping_rounds=5);
# save out model
bst.save_model(nameOfModel)

print ('finish training')


E:\Anaconda\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
E:\Anaconda\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


loading data end, start to boost trees
[0]	train-auc:0.663053
Will train until train-auc hasn't improved in 5 rounds.
[1]	train-auc:0.66604
[2]	train-auc:0.667768
[3]	train-auc:0.669458
[4]	train-auc:0.67073
[5]	train-auc:0.672265
[6]	train-auc:0.673055
[7]	train-auc:0.673859
[8]	train-auc:0.674466
[9]	train-auc:0.675054
[10]	train-auc:0.675574
[11]	train-auc:0.676196
[12]	train-auc:0.676949
[13]	train-auc:0.677537
[14]	train-auc:0.678157
[15]	train-auc:0.678649
[16]	train-auc:0.679425
[17]	train-auc:0.679844
[18]	train-auc:0.680404
[19]	train-auc:0.681105
[20]	train-auc:0.681423
[21]	train-auc:0.681988
[22]	train-auc:0.682421
[23]	train-auc:0.682925
[24]	train-auc:0.683341
[25]	train-auc:0.683676
[26]	train-auc:0.684228
[27]	train-auc:0.684833
[28]	train-auc:0.685242
[29]	train-auc:0.685734
[30]	train-auc:0.686219
[31]	train-auc:0.686732
[32]	train-auc:0.687039
[33]	train-auc:0.687384
[34]	train-auc:0.687809
[35]	train-auc:0.688126
[36]	train-auc:0.688602
[37]	train-auc:0.688976
[38]	

In [6]:
xgmat_test = xgboost.DMatrix( X_test, label=Y_test)
y_pred = bst.predict(xgmat_test)

In [7]:
predictions = [round(value) for value in y_pred]
X_test = pd.concat([X_test,dfNeeded_test],axis=1)
X_test['y_pred'] = y_pred

In [13]:
def getScore(df):
    df['rank_srch_id'] = df.groupby('srch_id').cumcount().add(1)
    df['gain'] = 5*df['booking_bool']
    df['gain'] = np.where(df['gain'] == 0, df['click_bool'], df['gain'])

    df['g/rank'] = df['gain']/df['rank_srch_id']
    gi_sum = df['g/rank'].sum()
    
    df['gain_sorted'] = list(df[['srch_id', 'booking_bool', 'click_bool', 'gain']].sort_values(by = ['srch_id','booking_bool','click_bool'], ascending = [True, False, False])['gain'])
    df['g/rank_sorted'] = df['gain_sorted']/df['rank_srch_id']
    gi_sorted_sum = df['g/rank_sorted'].sum() 
    score = gi_sum/gi_sorted_sum
    return score, gi_sum, gi_sorted_sum

In [14]:
score, gi_sum, gi_sorted_sum = getScore(X_test.sort_values(by=['srch_id', 'y_pred'], ascending = [True, False]).reset_index(drop=True))
print(score)

# evaluate predictions
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
Counter(predictions)

0.5866607673869075
Accuracy: 66.87%


Counter({1.0: 506455, 0.0: 981049})

Evaluation

In [10]:
dfTest = pd.read_csv("../data/testWithMean.csv")

In [11]:
clickingModel = xgboost.Booster({'nthread': 12})  # init model
clickingModel.load_model('clickingModel1may.model')  # load data


In [12]:
xgmat_test_submission = xgboost.DMatrix(dfTest[featuresToUse])
y_pred_clicking = clickingModel.predict(xgmat_test_submission)
dfTest['predictedPos'] = y_pred_clicking
dfTest = dfTest.sort_values(['srch_id','predictedPos'],ascending=[True, False])
dfSubmission = dfTest[['srch_id','prop_id']]
dfSubmission.to_csv("submissionTest1may.csv",index=False)